In [61]:
import pandas as pd
import numpy as np
import sys  
sys.path.insert(0, '../source/')
from preprocess_data_utils import *

In [129]:
#load real light curves
snIas=None
for i in range(11):
    snIas_str = '../data/testing/snIa_{}.csv'.format(i)
    print(snIas_str)
    snIa = pd.read_csv(snIas_str,sep="|").dropna(axis=1)
    if snIas is None:
        snIas = pd.concat([snIas, snIa],ignore_index=True)
    else:
        snIas = snIa

../data/testing/snIa_0.csv
../data/testing/snIa_1.csv
../data/testing/snIa_2.csv
../data/testing/snIa_3.csv
../data/testing/snIa_4.csv
../data/testing/snIa_5.csv
../data/testing/snIa_6.csv
../data/testing/snIa_7.csv
../data/testing/snIa_8.csv
../data/testing/snIa_9.csv
../data/testing/snIa_10.csv


In [130]:
snIas

,Unnamed: 0,objectId,jd,magpsf,sigmapsf
NaN,ZTF18aawmxdx,2.458462e+06,18.7827,0.101659,1
NaN,ZTF18aawmxdx,2.458465e+06,17.9839,0.065190,1
NaN,ZTF18aawmxdx,2.458465e+06,18.0191,0.058467,2
NaN,ZTF18aawmxdx,2.458469e+06,17.6659,0.058305,2
NaN,ZTF18aawmxdx,2.458472e+06,17.5561,0.056013,2
...,...,...,...,...,...
NaN,ZTF18adbhrjs,2.458495e+06,18.7096,0.080200,1
NaN,ZTF18adbhrjs,2.458503e+06,19.0431,0.106514,2
NaN,ZTF18adbhrjs,2.458509e+06,19.3554,0.209057,2
NaN,ZTF18adbhrjs,2.458512e+06,19.4559,0.118193,2


In [131]:
#rename columns
snIas.columns = ["id","time","flux","flux_err","band"]

In [132]:
snIas

,id,time,flux,flux_err,band
NaN,ZTF18aawmxdx,2.458462e+06,18.7827,0.101659,1
NaN,ZTF18aawmxdx,2.458465e+06,17.9839,0.065190,1
NaN,ZTF18aawmxdx,2.458465e+06,18.0191,0.058467,2
NaN,ZTF18aawmxdx,2.458469e+06,17.6659,0.058305,2
NaN,ZTF18aawmxdx,2.458472e+06,17.5561,0.056013,2
...,...,...,...,...,...
NaN,ZTF18adbhrjs,2.458495e+06,18.7096,0.080200,1
NaN,ZTF18adbhrjs,2.458503e+06,19.0431,0.106514,2
NaN,ZTF18adbhrjs,2.458509e+06,19.3554,0.209057,2
NaN,ZTF18adbhrjs,2.458512e+06,19.4559,0.118193,2


In [133]:
def check_lc_length(data):
    #different ids for different passbands
    data_cp = data.copy()
    data_cp['ob_p']=data.id+data.band.astype('str')
    #get dataframe with min and max mjd values per each object id
    group_by_mjd = data_cp.groupby(['id'])['time'].agg(['min', 'max']).rename(columns = lambda x : 'time_' + x).reset_index()
    group_by_mjd["time_diff"]=group_by_mjd.time_max-group_by_mjd.time_min
    return group_by_mjd

In [134]:
t_d=check_lc_length(snIas)


In [135]:
t_d.time_diff.describe()
#ufff muy muy poquitas obs

count    100.000000
mean      76.564685
std       89.617220
min        5.980046
25%       33.972297
50%       46.021863
75%       75.810431
max      465.796516
Name: time_diff, dtype: float64

In [136]:
snIa_tags = np.zeros(snIas.shape[0],dtype='int32')
snIas["tag"]=snIa_tags
snIas

,id,time,flux,flux_err,band,tag
NaN,ZTF18aawmxdx,2.458462e+06,18.7827,0.101659,1,0
NaN,ZTF18aawmxdx,2.458465e+06,17.9839,0.065190,1,0
NaN,ZTF18aawmxdx,2.458465e+06,18.0191,0.058467,2,0
NaN,ZTF18aawmxdx,2.458469e+06,17.6659,0.058305,2,0
NaN,ZTF18aawmxdx,2.458472e+06,17.5561,0.056013,2,0
...,...,...,...,...,...,...
NaN,ZTF18adbhrjs,2.458495e+06,18.7096,0.080200,1,0
NaN,ZTF18adbhrjs,2.458503e+06,19.0431,0.106514,2,0
NaN,ZTF18adbhrjs,2.458509e+06,19.3554,0.209057,2,0
NaN,ZTF18adbhrjs,2.458512e+06,19.4559,0.118193,2,0


In [137]:
enough_td = t_d.id[t_d.time_diff>50].values
snIas=snIas[snIas.id.isin(enough_td)]
len(snIas.id.unique())

43

In [112]:
snIbcs_str='../data/testing/snIbc.csv'
snIbcs = pd.read_csv(snIbcs_str,sep="|").dropna(axis=1)
#rename columns
snIbcs.columns = ["id","time","flux","flux_err","band"]
snIbcs

,id,time,flux,flux_err,band
0,ZTF18aakpigz,2.458280e+06,19.8569,0.167994,2
1,ZTF18aakpigz,2.458824e+06,17.8664,0.243286,1
2,ZTF18aakpigz,2.458828e+06,18.2403,0.092048,2
3,ZTF18aakpigz,2.458831e+06,18.2449,0.084126,1
4,ZTF18aakpigz,2.458834e+06,18.1861,0.106567,2
...,...,...,...,...,...
1166,ZTF20aagewqr,2.458865e+06,19.0348,0.090412,2
1167,ZTF20aagewqr,2.458875e+06,18.5037,0.086205,2
1168,ZTF20aagewqr,2.458875e+06,19.7588,0.164363,1
1169,ZTF20aagewqr,2.458876e+06,18.6594,0.089834,2


In [113]:
t_d=check_lc_length(snIbcs)
t_d.time_diff.describe()

count     51.000000
mean      88.544626
std      123.870959
min        1.055995
25%       27.852147
50%       45.981597
75%       96.800081
max      603.143588
Name: time_diff, dtype: float64

In [114]:
snIbc_tags = np.ones(snIbcs.shape[0],dtype='int32')
snIbcs["tag"]=snIbc_tags
snIbcs

,id,time,flux,flux_err,band,tag
0,ZTF18aakpigz,2.458280e+06,19.8569,0.167994,2,1
1,ZTF18aakpigz,2.458824e+06,17.8664,0.243286,1,1
2,ZTF18aakpigz,2.458828e+06,18.2403,0.092048,2,1
3,ZTF18aakpigz,2.458831e+06,18.2449,0.084126,1,1
4,ZTF18aakpigz,2.458834e+06,18.1861,0.106567,2,1
...,...,...,...,...,...,...
1166,ZTF20aagewqr,2.458865e+06,19.0348,0.090412,2,1
1167,ZTF20aagewqr,2.458875e+06,18.5037,0.086205,2,1
1168,ZTF20aagewqr,2.458875e+06,19.7588,0.164363,1,1
1169,ZTF20aagewqr,2.458876e+06,18.6594,0.089834,2,1


In [115]:
enough_td = t_d.id[t_d.time_diff>50].values
snIbcs=snIbcs[snIbcs.id.isin(enough_td)]
len(snIbcs.id.unique())

25

In [116]:
slsn_str='../data/testing/slsn1.csv'
slsn = pd.read_csv(slsn_str,sep="|").dropna(axis=1)
#rename columns
slsn.columns = ["id","time","flux","flux_err","band"]
slsn

,id,time,flux,flux_err,band
0,ZTF18abmasep,2.458344e+06,19.1595,0.154154,1
1,ZTF18abmasep,2.458345e+06,18.9690,0.099838,1
2,ZTF18abmasep,2.458347e+06,18.8206,0.096125,1
3,ZTF18abmasep,2.458348e+06,18.6596,0.092357,1
4,ZTF18abmasep,2.458350e+06,18.5943,0.102934,1
...,...,...,...,...,...
750,ZTF20aahbfmf,2.458892e+06,18.4552,0.078066,1
751,ZTF20aahbfmf,2.458892e+06,18.5348,0.079481,2
752,ZTF20aahbfmf,2.458893e+06,18.5723,0.088979,2
753,ZTF20aahbfmf,2.458895e+06,18.4279,0.059715,1


In [117]:
t_d=check_lc_length(slsn)
t_d.time_diff.describe()

count     19.000000
mean     103.857187
std      101.556170
min       26.847338
25%       56.382106
50%       72.901030
75%      105.724219
max      489.685741
Name: time_diff, dtype: float64

In [118]:
enough_td = t_d.id[t_d.time_diff>50].values
slsn=slsn[slsn.id.isin(enough_td)]
len(slsn.id.unique())

15

In [120]:
slsn_tags = np.repeat(5,slsn.shape[0])
slsn["tag"]=slsn_tags
slsn

/home/aibsen/miniconda3/envs/ztf-classification/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,time,flux,flux_err,band,tag
0,ZTF18abmasep,2.458344e+06,19.1595,0.154154,1,5
1,ZTF18abmasep,2.458345e+06,18.9690,0.099838,1,5
2,ZTF18abmasep,2.458347e+06,18.8206,0.096125,1,5
3,ZTF18abmasep,2.458348e+06,18.6596,0.092357,1,5
4,ZTF18abmasep,2.458350e+06,18.5943,0.102934,1,5
...,...,...,...,...,...,...
736,ZTF19abxgmzr,2.458783e+06,19.7309,0.231876,1,5
737,ZTF19abxgmzr,2.458786e+06,18.9695,0.140755,2,5
738,ZTF19abxgmzr,2.458786e+06,19.6247,0.171465,1,5
739,ZTF19abxgmzr,2.458790e+06,19.9352,0.287991,1,5


In [144]:
data = pd.concat([snIas,snIbcs,slsn],axis=0,sort=False)

In [145]:
data

,id,time,flux,flux_err,band,tag
NaN,ZTF18acdwazf,2.458428e+06,16.7246,0.031541,1,0
NaN,ZTF18acdwazf,2.458431e+06,16.4988,0.032495,1,0
NaN,ZTF18acdwazf,2.458441e+06,16.5983,0.031329,1,0
NaN,ZTF18acdwazf,2.458441e+06,16.7101,0.035568,2,0
NaN,ZTF18acdwazf,2.458444e+06,16.9187,0.036379,2,0
...,...,...,...,...,...,...
736.0,ZTF19abxgmzr,2.458783e+06,19.7309,0.231876,1,5
737.0,ZTF19abxgmzr,2.458786e+06,18.9695,0.140755,2,5
738.0,ZTF19abxgmzr,2.458786e+06,19.6247,0.171465,1,5
739.0,ZTF19abxgmzr,2.458790e+06,19.9352,0.287991,1,5


In [164]:
tags = data[['id','tag']]
tags.columns = ["id","type"]
tags=tags.drop_duplicates('id',keep='first')
tags

,id,type
NaN,ZTF18acdwazf,0
NaN,ZTF18acefhxx,0
NaN,ZTF18aceynvm,0
NaN,ZTF18achmxsx,0
NaN,ZTF18acmxkzj,0
...,...,...
494.0,ZTF19aawfbtg,5
593.0,ZTF19abaeyqw,5
613.0,ZTF19abpbopt,5
696.0,ZTF19abuyuwa,5


In [174]:
sample = data[['id','time','flux','flux_err','band']]
sample

,id,time,flux,flux_err,band
NaN,ZTF18acdwazf,2.458428e+06,16.7246,0.031541,1
NaN,ZTF18acdwazf,2.458431e+06,16.4988,0.032495,1
NaN,ZTF18acdwazf,2.458441e+06,16.5983,0.031329,1
NaN,ZTF18acdwazf,2.458441e+06,16.7101,0.035568,2
NaN,ZTF18acdwazf,2.458444e+06,16.9187,0.036379,2
...,...,...,...,...,...
736.0,ZTF19abxgmzr,2.458783e+06,19.7309,0.231876,1
737.0,ZTF19abxgmzr,2.458786e+06,18.9695,0.140755,2
738.0,ZTF19abxgmzr,2.458786e+06,19.6247,0.171465,1
739.0,ZTF19abxgmzr,2.458790e+06,19.9352,0.287991,1


In [175]:
def abmag_to_flux(m):
    exponent= (26.3-m)/2.5
    return np.power(10,exponent)

fluxes = abmag_to_flux(sample.flux.values)
sample['flux'] = fluxes
sample

/home/aibsen/miniconda3/envs/ztf-classification/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,time,flux,flux_err,band
NaN,ZTF18acdwazf,2.458428e+06,6763.320995,0.031541,1
NaN,ZTF18acdwazf,2.458431e+06,8326.835786,0.032495,1
NaN,ZTF18acdwazf,2.458441e+06,7597.662542,0.031329,1
NaN,ZTF18acdwazf,2.458441e+06,6854.250936,0.035568,2
NaN,ZTF18acdwazf,2.458444e+06,5656.138040,0.036379,2
...,...,...,...,...,...
736.0,ZTF19abxgmzr,2.458783e+06,424.267730,0.231876,1
737.0,ZTF19abxgmzr,2.458786e+06,855.460576,0.140755,2
738.0,ZTF19abxgmzr,2.458786e+06,467.864399,0.171465,1
739.0,ZTF19abxgmzr,2.458790e+06,351.495687,0.287991,1


In [176]:
lc_length = 50*128/700

In [177]:
lc_length #lets go with 10

9.142857142857142

In [194]:
def create_interpolated_vectors_(data, tags, length):
    obj_ids = tags.id.unique()
    obj_ids = np.arange(obj_ids.size)
    #different ids for different passbands
    data_cp = data.copy()
    data_cp['ob_p']=data.id+data.band.astype('str')
       
    #get dataframe with min and max mjd values per each object id
    group_by_mjd = data_cp.groupby(['id'])['time'].agg(['min', 'max']).rename(columns = lambda x : 'time_' + x).reset_index()
    merged = pd.merge(data_cp, group_by_mjd, how = 'left', on = 'id')
    #print(merged.id.unique().size == data_cp.id.unique().size)
    #scale mjd according to max mjd, min mjd and the desired length of the light curve (128)
    merged['scaled_time'] = (length - 1) * (merged['time'] - merged['time_min'])/(merged['time_max']-merged['time_min'])
    merged['count'] = 1
    merged['cc'] = merged.groupby(['ob_p'])['count'].cumcount()
    merged=merged.sort_values(['id','time'])
    #print(merged.id.unique().size==data_cp.id.unique().size)
    #print(data_cp.id.unique().size)
    
    #reshape df so that for each row there's one lightcurve (2 rows per obj) and each column is a point of it
    # there is two main columns also, for flux and for mjd
    unstack = merged[['ob_p', 'scaled_time', 'flux', 'cc']].set_index(['ob_p', 'cc']).unstack()
    #print(unstack.shape)
    #print(unstack.shape[0]== data_cp.id.unique().size*2)
    #transform above info into numpy arrays
    time_uns = unstack['scaled_time'].values[..., np.newaxis]
    flux_uns = unstack['flux'].values[..., np.newaxis]
    time_flux = np.concatenate((time_uns, flux_uns), axis =2)
    #print(flux_uns.shape)
    #print(time_uns.shape)
    #print(time_flux.shape)
    #create a mask to get points that are valid (not nan)
    nan_masks = ~np.isnan(time_flux)[:, :, 0]
    x = np.arange(length)
    
    n_lcs = time_flux.shape[0]
    #here we'll store interpolated lcs
    X = np.zeros((n_lcs, x.shape[0]))
    X_void = np.zeros((n_lcs, x.shape[0]))
    
    t=range(n_lcs)
    #print(n_lcs)
    #here we'll store the channels that tells us how far a point is from the nearest real point
    #interpolation
    for i in t: 
        if nan_masks[i].any():
            X[i] = np.interp(x, time_flux[i][:, 0][nan_masks[i]], time_flux[i][:, 1][nan_masks[i]])
        else:
            X[i] = np.zeros_like(x)
    # print(X.shape)
    #get distance for each point to nearest real point
    t=range(length)
    for i in t:
        X_void[:, i] = np.abs((unstack["scaled_time"] - i)).min(axis = 1).fillna(500)

    #reshape vectors so the ones belonging to the same object are grouped into 2 channels  
    #print(X)
    n_objs = int(X.shape[0]/2)
    X_per_band = X.reshape((n_objs,2,length)).astype(np.float32)
    # print(X_per_band.shape)
    X_void_per_band = X_void.reshape((n_objs,2,length)).astype(np.float32)
    # print(X_void_per_band.shape)
    vectors = np.concatenate((X_per_band,X_void_per_band),axis=1)
    # print(vectors.shape)
    return vectors, obj_ids, tags.type.values

x,ids,y=create_interpolated_vectors_(sample,tags,10)

In [195]:
x.shape

(83, 4, 10)

In [196]:
ids.shape

(83,)

In [197]:
y.shape

(83,)

In [199]:
dataset = {"X":x,"ids":ids,"Y": y}
def save_vectors_(dataset, outputFile):
    hf=h5py.File(outputFile,'w')
    print("writing X")
    hf.create_dataset('X',data=dataset['X'])
    print("writing ids")
    hf.create_dataset('ids',data=dataset['ids'])
    print("writing Y")
    hf.create_dataset('Y',data=dataset['Y'])
    hf.close()

save_vectors_(dataset, "ztf_snIa_bc_sl_50_fluxes_zp_26.h5")

writing X
writing ids
writing Y
